In [22]:
import pandas as pd
import os
import numpy as np


In [23]:
# File paths (replace with your actual file paths)
file_paths = ['./jobs_dataset/Data_Job_NY.csv', './jobs_dataset/Data_Job_SF.csv', './jobs_dataset/Data_Job_TX.csv', './jobs_dataset/Data_Job_WA.csv']

# Load CSV files into DataFrames
dataframes = [pd.read_csv(file_path) for file_path in file_paths]

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the shape of the combined DataFrame
print(f"Combined Dataset Shape: {combined_df.shape}")


Combined Dataset Shape: (3324, 12)


In [24]:
# Display the first few rows
display(combined_df.head())

# Show summary info
combined_df.info()

# Check for missing values
print(combined_df.isnull().sum())


,Job_title,Company,State,City,Min_Salary,Max_Salary,Job_Desc,Industry,Rating,Date_Posted,Valid_until,Job_Type
0,Chief Marketing Officer (CMO),National Debt Relief,NY,New York,-1,-1,Who We're Looking For:\n\nThe Chief Marketing ...,Finance,4.0,2020-05-08,2020-06-07,FULL_TIME
1,Registered Nurse,Queens Boulevard Endoscopy Center,NY,Rego Park,-1,-1,"Queens Boulevard Endoscopy Center, an endoscop...",NaN,3.0,2020-04-25,2020-06-07,FULL_TIME
2,Dental Hygienist,Batista Dental,NJ,West New York,-1,-1,Part-time or Full-timedental hygienist positio...,NaN,NaN,2020-05-02,2020-06-07,PART_TIME
3,Senior Salesforce Developer,National Debt Relief,NY,New York,44587,82162,Principle Duties & Responsibilities:\n\nAnalyz...,Finance,4.0,2020-05-08,2020-06-07,FULL_TIME
4,"DEPUTY EXECUTIVE DIRECTOR, PROGRAM AND LEGAL A...",National Advocates for Pregnant Women,NY,New York,125410,212901,"For FULL Job Announcement, visit our website: ...",NaN,NaN,2020-04-28,2020-06-07,FULL_TIME


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3324 entries, 0 to 3323
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Job_title    3324 non-null   object 
 1   Company      3324 non-null   object 
 2   State        3322 non-null   object 
 3   City         3318 non-null   object 
 4   Min_Salary   3324 non-null   int64  
 5   Max_Salary   3324 non-null   int64  
 6   Job_Desc     3324 non-null   object 
 7   Industry     2700 non-null   object 
 8   Rating       2849 non-null   float64
 9   Date_Posted  3324 non-null   object 
 10  Valid_until  3324 non-null   object 
 11  Job_Type     3324 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 311.8+ KB
Job_title        0
Company          0
State            2
City             6
Min_Salary       0
Max_Salary       0
Job_Desc         0
Industry       624
Rating         475
Date_Posted      0
Valid_until      0
Job_Type         0
dtype: int64


In [25]:
# Replace -1 in Min_Salary and Max_Salary with NaN
combined_df['Min_Salary'] = combined_df['Min_Salary'].replace(-1, np.nan)
combined_df['Max_Salary'] = combined_df['Max_Salary'].replace(-1, np.nan)

# Display the updated missing value counts
print(combined_df[['Min_Salary', 'Max_Salary']].isnull().sum())
print(f"Combined Dataset Shape: {combined_df.shape}")




Min_Salary    1092
Max_Salary    1092
dtype: int64
Combined Dataset Shape: (3324, 12)


In [26]:
# Remove rows where Rating is invalid (not between 0 and 5)
combined_df = combined_df[(combined_df['Rating'] >= 0) & (combined_df['Rating'] <= 5)]
print(f"Combined Dataset Shape: {combined_df.shape}")


# Fill missing ratings with the average rating
combined_df['Rating'] = combined_df['Rating'].fillna(combined_df['Rating'].mean())
print(f"Combined Dataset Shape: {combined_df.shape}")


# Display rating statistics
print(combined_df['Rating'].describe())
print(f"Combined Dataset Shape: {combined_df.shape}")



Combined Dataset Shape: (2849, 12)
Combined Dataset Shape: (2849, 12)
count    2849.000000
mean        3.837733
std         0.626777
min         1.000000
25%         3.500000
50%         3.900000
75%         4.200000
max         5.000000
Name: Rating, dtype: float64
Combined Dataset Shape: (2849, 12)


In [27]:
# Convert Date_Posted and Valid_until to datetime format
combined_df['Date_Posted'] = pd.to_datetime(combined_df['Date_Posted'], format='%Y-%m-%d', errors='coerce')
combined_df['Valid_until'] = pd.to_datetime(combined_df['Valid_until'], format='%Y-%m-%d', errors='coerce')

print(f"Combined Dataset Shape: {combined_df.shape}")


# Drop rows with invalid dates
combined_df = combined_df.dropna(subset=['Date_Posted', 'Valid_until'])
print(f"Combined Dataset Shape: {combined_df.shape}")


# Check for any remaining issues in date columns
print(combined_df[['Date_Posted', 'Valid_until']].isnull().sum())


Combined Dataset Shape: (2849, 12)
Combined Dataset Shape: (2849, 12)
Date_Posted    0
Valid_until    0
dtype: int64


In [28]:
# Strip whitespace from text columns
text_columns = ['Job_title', 'Company', 'State', 'City', 'Industry', 'Job_Type']
combined_df[text_columns] = combined_df[text_columns].apply(lambda x: x.str.strip())

# Convert Job_Type to uppercase for consistency
combined_df['Job_Type'] = combined_df['Job_Type'].str.upper()

# Display unique job types
print(combined_df['Job_Type'].unique())


['FULL_TIME' 'PART_TIME' 'OTHER' 'INTERN' 'CONTRACTOR' 'TEMPORARY']


In [ ]:
# Create directory if it doesn't exist
os.makedirs('./jobs_dataset/cleaned_dataset', exist_ok=True)

# Save the cleaned dataset as a CSV file
cleaned_file_path = './jobs_dataset/cleaned_dataset/cleaned_jobs_data.csv'
combined_df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved at: {cleaned_file_path}")

Cleaned dataset saved at: ./jobs_dataset/cleaned_dataset/cleaned_jobs_data.csv
